In [2]:
# 🔧 Bibliotecas principais
import pandas as pd
import numpy as np
import json
import gzip
import tarfile
import os

# 📦 URLs dos dados do case iFood
order_url = 'https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz'
consumer_url = 'https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz'
restaurant_url = 'https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz'
ab_test_url = 'https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz' # Corrected URL

# ⬇️ Baixando arquivos (apenas no Google Colab)
!wget $order_url -O order.json.gz
!wget $consumer_url -O consumer.csv.gz
!wget $restaurant_url -O restaurant.csv.gz
!wget $ab_test_url -O ab_test.tar.gz

# 📂 Extraindo o arquivo do teste A/B
with tarfile.open('ab_test.tar.gz', 'r:gz') as tar:
    tar.extractall()

In [2]:
# 📥 Leitura dos arquivos
import gzip
import pandas as pd
import json
import os

import io

orders = []
with gzip.open('order.json.gz', 'rb') as f:
    for i, line in enumerate(f):
        if i % 100000 == 0:
            print(f"Lendo linha: {i}")
        if i == 1000000:  # Limite opcional (ex: 1 milhão de linhas)
            break
        orders.append(json.loads(line))
orders = pd.DataFrame(orders)


consumers = pd.read_csv('consumer.csv.gz')
restaurants = pd.read_csv('restaurant.csv.gz')

ab_test_file = [f for f in os.listdir() if 'ab_test' in f and f.endswith('.csv')][0]
ab_test = pd.read_csv(ab_test_file, encoding='latin-1') # Added encoding

Lendo linha: 0
Lendo linha: 100000
Lendo linha: 200000
Lendo linha: 300000
Lendo linha: 400000
Lendo linha: 500000
Lendo linha: 600000
Lendo linha: 700000
Lendo linha: 800000
Lendo linha: 900000
Lendo linha: 1000000


In [3]:
# 👥 Mesclando usuários e grupo de teste
consumers_ab = pd.merge(consumers, ab_test, on='customer_id', how='inner')

# 🛒 Mesclando com pedidos
df = pd.merge(orders, consumers_ab, on='customer_id', how='inner')

# 🔎 Visualizando
print("DataFrame final:", df.shape)
df.head()


KeyError: 'customer_id'

In [4]:
# List files in the current directory to identify the extracted ab_test file
!ls

# Read the ab_test file to inspect its contents and column names
ab_test_file = [f for f in os.listdir() if 'ab_test' in f and f.endswith('.csv')][0]
try:
    ab_test_content = pd.read_csv(ab_test_file, encoding='latin-1')
    print("\nContent of the ab_test file:")
    display(ab_test_content.head())
    print("\nColumns in the ab_test file:")
    print(ab_test_content.columns)
except Exception as e:
    print(f"Error reading ab_test file: {e}")

ab_test_ref.csv  consumer.csv.gz  restaurant.csv.gz
ab_test.tar.gz	 order.json.gz	  sample_data

Content of the ab_test file:


,Unnamed: 0



Columns in the ab_test file:
Index(['Unnamed: 0'], dtype='object')
